# Python Implementation of RSA, Digital Signature and Certificates
> Implementation of RSA, Digital Signature and Certificates without divining too deep into the Mathematics; then list out key math principles for further reference. In this way, one can understand *what* and *how* as a context before drilling into *why*.

- toc: true
- branch: master
- badges: false
- comments: true
- categories: [jupyter, python, math]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

## Introduction

Information is power. However, to transmit info is tricky. At least 2 issues need to be solved in information transmission: **Security** and **Integrity**.

- **Security (RSA)**

No one can see the naked message other than the one intended.

- **Integrity (Digital Signature)**

No one shall replace the message with another dummy one in the middle of transimission;  
If it happened, however, intended message receiver shall be able to find out.

## Rivest, Shamir and Adleman (RSA) Cryptosystem

![rsa](rsa_imgs/bg2013062702.jpg)
@MIT

### Helper Functions

In [1]:
#collapse-hide
import random
import base64

#### Check If a Number is Prime

In [2]:
#collapse-show
def check_prime(num):
    """Check if num is prime
    
    """
    # prime numbers are greater than 1
    if num > 1:
        # check for factors
        # TODO: refactoring to parallel
        for i in range(2, num):
            if (num % i) == 0:
                return False
        else:
            return True
    # if input number is less than
    # or equal to 1, it is not prime
    else:
        return False

#### Modular Multiplicative Inverse Function

In [3]:
#collapse-show
# https://stackoverflow.com/a/9758173/3317548
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)


def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception("modular inverse does not exist")
    else:
        aa = x % m
        bb = int((1 - a * aa) / m)
        return aa, bb

#### List All Prime Numbers Smaller Than a Value

In [4]:
#collapse-show
# https://stackoverflow.com/q/2068372/3317548
def find_prime(n):
    """ Returns  a list of primes < n """
    sieve = [True] * (n // 2)
    for i in range(3, int(n ** 0.5) + 1, 2):
        if sieve[i // 2]:
            sieve[i * i // 2 :: i] = [False] * ((n - i * i - 1) // (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, n // 2) if sieve[i]]

###  Encryption & Decryption Process

- Receiver (Richael) generate a pair of keys: **public** & **private**
- She keeps **private** key to herself confidentially
- She sends multiple copies of **public** key to every of her friends who need to send her message
- Any one got Richael's **public** key can send her message now. For instance, Stephen:  
    - Stephen'd like to send the original message: "Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 :)", to Richael.
    - Stephen'd encrypt the message with Richaels **public** key into something looks random, e.g.  "MHgxNTMzCjB4MmFmNgoweGFiOA...YWE0CjB4MWZlMwoweDIzMGY="
    - Stephen'd send that "random" stuff to Richael.
- After receiving the "random" stuff from Stephen, Richael uses her own **private** key to decrypt it into orignal message "Hello Richael. How are you?!"
- Only with Richael's **private** key, the "random" stuff, produced with Richael's **public** key, can be decrpted. Unless Richael shares her **private** key, which she shall never do, no one other than Richael herself can obtain the original message "Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 :)"
- If Richael'd like to answer to Stephen, she must obtain Stephen's **public** key.

#### Key Pair: Public & Private (Richael's Task)

- Randomly pick 2 prime numbers (larger the better): **p** & **q** 
- Got their muliplication: **n=pxq**
- Got the Euler number: **phi=(p-1)x(q-1)**
- Randomly find a prime number in the range of 1 to **phi** (larger the better): **e**
- Find the Modular Multiplicative Inverse of **e** relative to **phi**: **k**
- public key is: **(n, e)**
- private key is: **(n, k)**

In [33]:
# prime numbers
p = random.choice(list(find_prime(1024)))
q = random.choice(list(find_prime(1024)))
assert check_prime(p)
assert check_prime(q)
n = p * q
p, q, n, len(bin(n)), bin(n)

(193, 883, 170419, 20, '0b101001100110110011')

In [34]:
# Euler number
phi = (p - 1) * (q - 1)
phi

169344

In [35]:
e = random.choice(find_prime(phi))
assert check_prime(e)
e

29567

In [37]:
k = modinv(e, phi)[0]
k

26879

In [38]:
public_key, private_key = (n, e), (n, k)
public_key, private_key

((170419, 29567), (170419, 26879))

#### Encryption & Decryption

- Original message to unicode numbers: **msg**
- Encrypt message: **msg_encrypt = (msg^e)%n**
- Decrypt message: **msg = (msg_encrypt^k)%n**

#### Origial Message (Stephen's Task)

In [67]:
#collapse-output
# unicode
msg = [ord(i) for i in "Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 :)"]
msg

[72,
 101,
 108,
 108,
 111,
 32,
 82,
 105,
 99,
 104,
 97,
 101,
 108,
 46,
 32,
 76,
 101,
 116,
 39,
 115,
 32,
 109,
 101,
 101,
 116,
 32,
 97,
 116,
 32,
 54,
 58,
 50,
 53,
 32,
 112,
 109,
 32,
 116,
 111,
 109,
 111,
 114,
 114,
 111,
 119,
 32,
 97,
 116,
 32,
 116,
 104,
 101,
 32,
 97,
 105,
 114,
 112,
 111,
 114,
 116,
 32,
 71,
 97,
 116,
 101,
 53,
 44,
 32,
 84,
 101,
 114,
 109,
 105,
 110,
 97,
 108,
 32,
 50,
 32,
 58,
 41]

#### Encrption (Stephen's Task)

In [68]:
#collapse-output
# encrption
[(val ** public_key[1]) % public_key[0] for val in msg]

[169001,
 140011,
 129587,
 129587,
 126069,
 169062,
 132937,
 13635,
 118348,
 33595,
 38409,
 140011,
 129587,
 14689,
 169062,
 97239,
 140011,
 120437,
 154692,
 81300,
 169062,
 98901,
 140011,
 140011,
 120437,
 169062,
 38409,
 120437,
 169062,
 71964,
 92264,
 16571,
 60074,
 169062,
 72070,
 98901,
 169062,
 120437,
 126069,
 98901,
 126069,
 750,
 750,
 126069,
 116705,
 169062,
 38409,
 120437,
 169062,
 120437,
 33595,
 140011,
 169062,
 38409,
 13635,
 750,
 72070,
 126069,
 750,
 120437,
 169062,
 134415,
 38409,
 120437,
 140011,
 60074,
 1487,
 169062,
 65921,
 140011,
 750,
 98901,
 13635,
 32710,
 38409,
 129587,
 169062,
 16571,
 169062,
 92264,
 58013]

In [69]:
# hex encode or something else
encry_msg = '\n'.join([hex((val ** public_key[1]) % public_key[0]) for val in msg])
encry_msg

'0x29429\n0x222eb\n0x1fa33\n0x1fa33\n0x1ec75\n0x29466\n0x20749\n0x3543\n0x1ce4c\n0x833b\n0x9609\n0x222eb\n0x1fa33\n0x3961\n0x29466\n0x17bd7\n0x222eb\n0x1d675\n0x25c44\n0x13d94\n0x29466\n0x18255\n0x222eb\n0x222eb\n0x1d675\n0x29466\n0x9609\n0x1d675\n0x29466\n0x1191c\n0x16868\n0x40bb\n0xeaaa\n0x29466\n0x11986\n0x18255\n0x29466\n0x1d675\n0x1ec75\n0x18255\n0x1ec75\n0x2ee\n0x2ee\n0x1ec75\n0x1c7e1\n0x29466\n0x9609\n0x1d675\n0x29466\n0x1d675\n0x833b\n0x222eb\n0x29466\n0x9609\n0x3543\n0x2ee\n0x11986\n0x1ec75\n0x2ee\n0x1d675\n0x29466\n0x20d0f\n0x9609\n0x1d675\n0x222eb\n0xeaaa\n0x5cf\n0x29466\n0x10181\n0x222eb\n0x2ee\n0x18255\n0x3543\n0x7fc6\n0x9609\n0x1fa33\n0x29466\n0x40bb\n0x29466\n0x16868\n0xe29d'

In [70]:
# base64 encode
encry_msg = base64.b64encode(encry_msg.encode('utf8'))
encry_msg

b'MHgyOTQyOQoweDIyMmViCjB4MWZhMzMKMHgxZmEzMwoweDFlYzc1CjB4Mjk0NjYKMHgyMDc0OQoweDM1NDMKMHgxY2U0YwoweDgzM2IKMHg5NjA5CjB4MjIyZWIKMHgxZmEzMwoweDM5NjEKMHgyOTQ2NgoweDE3YmQ3CjB4MjIyZWIKMHgxZDY3NQoweDI1YzQ0CjB4MTNkOTQKMHgyOTQ2NgoweDE4MjU1CjB4MjIyZWIKMHgyMjJlYgoweDFkNjc1CjB4Mjk0NjYKMHg5NjA5CjB4MWQ2NzUKMHgyOTQ2NgoweDExOTFjCjB4MTY4NjgKMHg0MGJiCjB4ZWFhYQoweDI5NDY2CjB4MTE5ODYKMHgxODI1NQoweDI5NDY2CjB4MWQ2NzUKMHgxZWM3NQoweDE4MjU1CjB4MWVjNzUKMHgyZWUKMHgyZWUKMHgxZWM3NQoweDFjN2UxCjB4Mjk0NjYKMHg5NjA5CjB4MWQ2NzUKMHgyOTQ2NgoweDFkNjc1CjB4ODMzYgoweDIyMmViCjB4Mjk0NjYKMHg5NjA5CjB4MzU0MwoweDJlZQoweDExOTg2CjB4MWVjNzUKMHgyZWUKMHgxZDY3NQoweDI5NDY2CjB4MjBkMGYKMHg5NjA5CjB4MWQ2NzUKMHgyMjJlYgoweGVhYWEKMHg1Y2YKMHgyOTQ2NgoweDEwMTgxCjB4MjIyZWIKMHgyZWUKMHgxODI1NQoweDM1NDMKMHg3ZmM2CjB4OTYwOQoweDFmYTMzCjB4Mjk0NjYKMHg0MGJiCjB4Mjk0NjYKMHgxNjg2OAoweGUyOWQ='

#### Decryption (Richael's Task)

In [71]:
#collapse-output
# base64 decode and hex decode
[int(i, 16) for i in base64.b64decode(encry_msg).decode('utf8').split('\n')]

[169001,
 140011,
 129587,
 129587,
 126069,
 169062,
 132937,
 13635,
 118348,
 33595,
 38409,
 140011,
 129587,
 14689,
 169062,
 97239,
 140011,
 120437,
 154692,
 81300,
 169062,
 98901,
 140011,
 140011,
 120437,
 169062,
 38409,
 120437,
 169062,
 71964,
 92264,
 16571,
 60074,
 169062,
 72070,
 98901,
 169062,
 120437,
 126069,
 98901,
 126069,
 750,
 750,
 126069,
 116705,
 169062,
 38409,
 120437,
 169062,
 120437,
 33595,
 140011,
 169062,
 38409,
 13635,
 750,
 72070,
 126069,
 750,
 120437,
 169062,
 134415,
 38409,
 120437,
 140011,
 60074,
 1487,
 169062,
 65921,
 140011,
 750,
 98901,
 13635,
 32710,
 38409,
 129587,
 169062,
 16571,
 169062,
 92264,
 58013]

In [72]:
# RSA decode
decry_msg = [
    int(i, 16) ** private_key[1] % private_key[0]
    for i in base64.b64decode(encry_msg).decode("utf8").split("\n")
]
print(decry_msg)
print("".join([chr(i) for i in decry_msg]))

[72, 101, 108, 108, 111, 32, 82, 105, 99, 104, 97, 101, 108, 46, 32, 76, 101, 116, 39, 115, 32, 109, 101, 101, 116, 32, 97, 116, 32, 54, 58, 50, 53, 32, 112, 109, 32, 116, 111, 109, 111, 114, 114, 111, 119, 32, 97, 116, 32, 116, 104, 101, 32, 97, 105, 114, 112, 111, 114, 116, 32, 71, 97, 116, 101, 53, 44, 32, 84, 101, 114, 109, 105, 110, 97, 108, 32, 50, 32, 58, 41]
Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 :)


### Why is this secure?

![Euler](rsa_imgs/euler.png)
@Euler  
*He changed the world and he is now everywhere; Remeber Euler and everything about him, otherwise you'd fail at some stage of your life, some math exams for sure.*

- If another friend of Richael, say Tom, would like to know what Stephen sent to Richael, without asking Stephen or Richael.
- Another guy, say Joseph, was eavesdropping the network communication.
- Either Tom or Joseph would already have access to information of: 
    - **n** and **e** from public key because Richael broadcast it on the network to her friends.
    - **msg_encrypt** because Stephen broadcast it on the network for Richael to retrieve.
- In order to decrypt **msg_encrypt**, they need **private key** **(n, k)**;
- They already have **n** so only need to figure out **k**.
- **k** was generated with **e** and **phi**.
- They know **e**. So they only need to know *phi*. 
- They need **p** and **q** to know **phi**
- They know **n** which is the multiplication of prime numbers **p** and **q**
- Tom or Joseph need to do **Prime Factorization** as **n=pxq**
- Given **n** is large enough (1024 bits in nowadays common sense), Tom and Joseph would had been long gone before a modern computer, even they have access to [Fugaku](https://en.wikipedia.org/wiki/TOP500), could manage to figure out which 2 prime numebrs Richael picked at the first place, let along about by "6:25 pm tomrrow".
- Richael may choose to update her key pairs frequently (change to different **p** and **q**) for the next message; this means Tom and Joseph can never catch up without a [Quantum Computer](https://www.newscientist.com/article/2227387-quantum-computer-sets-new-record-for-finding-prime-number-factors/).

### Understand Your Computer

Now if go to `~/.ssh`, you shall identify a number of different key files. Some of them are private key whilst others are public key; some are generated by OpenSSH whilst others by OpenSSL. They are also encoded by ASN.1 format among some variates.

1. You can convert it into `*.PEM` file via:

`ssh-keygen -f YOUR_KEY.pub -e -m pem > YOUR_KEY.pub.pem`

2. Then inspect the content via:

```python
from Crypto.PublicKey import RSA
from base64 import b64decode

pem_key = b"MII<DO NOT SHARE PRIVATE KEY>="
key = b64decode(pem_key)
keyPriv = RSA.importKey(key)
# key now has all the components of the private
print(keyPriv.keydata)
print(keyPriv.n, keyPriv.e, keyPriv.d, keyPriv.p, keyPriv.q, keyPriv.u)

assert keyPriv.p * keyPriv.q == keyPriv.n
```

For public key `*.pub.pem`, it has only `keyPriv.n` and `keyPriv.e`.   
See: https://www.dlitz.net/software/pycrypto/api/2.6/Crypto.PublicKey.RSA-module.html

3. Similarly, if you check `less ~/.ssh/known_hosts`, you shall see a number of lines of records, composing of:  

`<domain_name,ip_address> <algorithm, e.g. ssh-rsa> <public_key>`  

You can convert those `public_key` into `*.pem` like above and use the code snippet to check its content. You shall got `n` and `e`.

## Digital Signature

There are 2 problems with the encryption and decryption process described above:  

1. What if Joseph pretends to be Stephen, and sends her another message encrypted with Richaels **public key**, saying "Hello Richael. Let's meet at 6:25 am the day after tomorrow at the train station Gate 1 :)".

*Answer*: this is solved by Digital Signature

2. Joseph could totally generate his own key pair and pretend to be Richael. When Richael's friends receive "her" **public key**, how do they know it truly comes from Richael, rather than Joseph?

*Answer*: this is solved by Certificate

## Reference

- https://blog.vrypan.net/2013/08/28/public-key-cryptography-for-non-geeks/
- https://www.ruanyifeng.com/blog/2013/06/rsa_algorithm_part_one.html
- https://us-cert.cisa.gov/ncas/tips/ST04-018#:~:text=Digital%20signatures%20work%20by%20proving,using%20the%20sender's%20private%20key.